Rand Agent Table with RNN for Time Series:

MODEL 4


* For agents with type Car only
* For all scenes 
* 20 most frequent agents only
* zero-padded (front padding included)

Look at this for more info on multiple parallel series with RNN: https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [1]:
import keras
import tensorflow as tf

Using TensorFlow backend.
E:\app\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\app\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\app\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\app\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 

In [2]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout

In [3]:
import pandas as pd
import numpy as np

In [4]:
folderpath = "."
file = folderpath + "/lyftlong/rand_agents_table0.csv"
at = pd.read_csv(file)
file = folderpath + "/lyftlong/rand_frames_table1.csv"
ft = pd.read_csv(file)
file = folderpath +  "/lyftlong/rand_scenes_table1.csv"
st = pd.read_csv(file)

In [5]:
at = at.merge(ft[['frame_index', 'scene_index']], on='frame_index')

In [6]:
at = at[at.PERCEPTION_LABEL_CAR == 1] 
# FEATURE: filter to only types with Car; uncomment the above to include all agent types

In [7]:
atcols = ['centroid_x', 'centroid_y', 'track_id', 'frame_index', 'scene_index']

In [8]:
atrnn = at[atcols]

In [9]:
scene_indices = pd.unique(at['scene_index'])

In [10]:
atrnn1 = atrnn[atrnn.scene_index == scene_indices[10]]

In [12]:
atrnn1

,centroid_x,centroid_y,track_id,frame_index,scene_index
186206,692.800476,-1370.700928,1,306162,1233
186207,725.804993,-1385.745117,2,306162,1233
186208,689.145996,-1375.814331,3,306162,1233
186209,676.625244,-1360.584961,4,306162,1233
186210,647.278381,-1374.638184,5,306162,1233
...,...,...,...,...,...
202597,960.734802,-1547.492065,3,306406,1233
202680,961.751038,-1548.127563,3,306407,1233
202760,962.582214,-1548.629639,3,306408,1233
202844,963.673706,-1549.282715,3,306409,1233


In [13]:
agent_parallel_input_series = pd.DataFrame()
apis = agent_parallel_input_series
# apis will be the dataframe that we will partiton for the rnn data table information

atrnn1s = atrnn1 #.sort_values(['track_id', 'frame_index'])

track_ids = pd.unique(atrnn1s['track_id'])
NUM_ENTITIES = len(track_ids)

In [14]:
# FEATURE: We will only take the 20 most frequent track ids. Set N_MOST_FREQUENT to NUM_ENTITIES if you don't want that

freq_track_ids = atrnn1s.groupby(['track_id']).count().reset_index()

N_MOST_FREQUENT = 20 # NUM_ENTITIES
assert N_MOST_FREQUENT <= NUM_ENTITIES
def get_n_most_frequent(n: int = NUM_ENTITIES):
    """ get's track ids of the n most frequent entities in a scene."""
    global freq_track_ids
    freq_track_ids = freq_track_ids.sort_values('frame_index', ascending=False)
    return freq_track_ids['track_id'].iloc[:20].values
track_ids = get_n_most_frequent(N_MOST_FREQUENT)

In [15]:
apis_cols = ['centroid_x', 'centroid_y'] + ['frame_index'] #  ['velocity_x', 'velocity_y'] +  ['frame_index'] + ['area']

In [16]:
apis_frame_indices = pd.unique(atrnn1['frame_index'])

In [17]:
apis['frame_index'] = apis_frame_indices # added to the apis table so that each agent can be merged 

In [18]:
for i in range(len(track_ids)):
    if i%100 == 0:
        print("{0} of {1}".format(i, len(track_ids)))
    ti = track_ids[i]
    # for each track id, get it's subset in atrnn1s
    atrnn1sti = atrnn1s[atrnn1s.track_id == ti]
    # get these columns only
    atrnn1sti = atrnn1sti[apis_cols]
    # rename the columns by labelling it with it's track id
    atrnn1sti = atrnn1sti.add_prefix('ti{0}_'.format(str(ti)))
    # undo apis index's renaming for joining to apis
    atrnn1sti.rename(columns = {"ti{0}_frame_index".format(str(ti)): "frame_index"},  
                     inplace = True) 
    # LEFT OUTER JOIN with APIS, atrnn1sti
    apis = pd.merge(apis, atrnn1sti, on='frame_index', how='left')

0 of 20


In [19]:
apis = apis.drop(['frame_index'], axis=1)

In [20]:
apis
# if you set N_MOST_FREQUENT to 20, there should be 40 columns only

,ti3_centroid_x,ti3_centroid_y,ti919_centroid_x,ti919_centroid_y,ti1_centroid_x,ti1_centroid_y,ti8_centroid_x,ti8_centroid_y,ti184_centroid_x,ti184_centroid_y,...,ti13_centroid_x,ti13_centroid_y,ti739_centroid_x,ti739_centroid_y,ti104_centroid_x,ti104_centroid_y,ti143_centroid_x,ti143_centroid_y,ti780_centroid_x,ti780_centroid_y
0,689.145996,-1375.814331,NaN,NaN,692.800476,-1370.700928,651.313904,-1349.105713,NaN,NaN,...,664.939941,-1326.213501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,691.124390,-1377.168457,NaN,NaN,692.172546,-1370.346069,651.312134,-1349.097168,NaN,NaN,...,664.962952,-1326.096436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,692.187500,-1377.882568,NaN,NaN,691.065918,-1369.599731,651.319641,-1349.092651,NaN,NaN,...,664.859680,-1325.897095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,693.238525,-1378.630859,NaN,NaN,690.384094,-1369.178833,651.310242,-1349.093262,NaN,NaN,...,664.943237,-1325.433472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,694.389648,-1379.216675,NaN,NaN,689.790222,-1368.812622,651.306213,-1349.086426,NaN,NaN,...,665.131287,-1325.381958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,960.734802,-1547.492065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,961.751038,-1548.127563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,962.582214,-1548.629639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,963.673706,-1549.282715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# FEATURE: replace all NaN's with zeros
apis = apis.fillna(0)

In [22]:
atrnn1 = apis

In [23]:
def make_partitions(ar: np.array, step: int = 3) -> np.array:
    len_ar = len(ar)
    partitions = np.array([])
    for i in range(len_ar - step):
        X_partition = ar[i: i + step]
        y_partition = ar[i + step]
        partition = dict()
        partition['X'] = ((X_partition))
        partition['y'] = ((y_partition))
        partitions = np.append(partitions, partition)
    return partitions

In [24]:
NSTEPS = 3

In [25]:
partitions = make_partitions(np.array(atrnn1), NSTEPS)

In [26]:
TRAIN_PROPORTION = 0.8
TRAIN_PROPORTION_INDEXER = int(len(atrnn1) * 0.8)

In [27]:
X_train = []
y_train = []
X_test = []
y_test = []
for i in range(len(partitions)): # [:TRAIN_PROPORTION_INDEXER]:
    partition = partitions[i]
    if i <= TRAIN_PROPORTION_INDEXER:
        X_train.append(partition['X'])
        y_train.append(partition['y'])
    else:
        X_test.append(partition['X'])
        y_test.append(partition['y'])        
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [28]:
n_steps = NSTEPS
n_features = len(atrnn1.columns)

act = 'tanh'
act1 = 'relu'
recact = 'sigmoid'
retseq = True

#### define model
model = Sequential()
model.add(LSTM(100, activation=act, recurrent_activation=recact, return_sequences=retseq, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation=act))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=300, verbose=0)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [30]:
model = Sequential()
model.add(LSTM(100,input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(50, activation = 'tanh', return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(25, activation = 'tanh', return_sequences = False))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1]))
model.compile(loss='mse', optimizer='adam', metrics = ['accuracy'])

model_info = model.fit(X_train, y_train, epochs=300, verbose=0)
print(model_info)

In [31]:
x_inp = X_test[0]
y_targ = y_test[0]

In [32]:
x_inp = X_test[0]
y_targ = y_test[0]

x_inp = x_inp = x_inp.reshape((1, n_steps, n_features))
x_inp.shape

y_hat = model.predict(x_inp, verbose=0)[0]

In [33]:
print(y_targ)

[  917.27105713 -1519.97790527   853.05664062 -1472.29333496
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.        ]


In [34]:
print(y_hat)

[ 51.543743 -53.168907  53.91406  -54.770542  37.229446 -38.182804
  34.935226 -37.479065  35.57664  -37.938747  33.61367  -34.257706
  38.380817 -40.668217  36.106056 -37.173256  30.801292 -34.159603
  29.91092  -31.729055  42.612858 -46.81182   29.092216 -32.153233
  35.73994  -36.446255  34.99002  -38.578842  30.813421 -32.25333
  27.934166 -30.909315  41.476856 -44.446095  29.45362  -32.6945
  28.24295  -32.86296   41.625008 -44.787815]


In [35]:
model_info = model.evaluate(X_test, y_test, batch_size=100)

46/46 [==============================] - 0s 4ms/sample - loss: 111571.0547 - acc: 0.3913


## SECTION LIMIT

In [ ]:
y_hat = model.predict(x_inp, verbose=0)[0]

In [ ]:
y_hat.astype(int)

In [ ]:
8.4155383e+02, -1.4250477e+03

In [ ]:
y_targ, 

In [ ]:
(1/len(y_hat))*np.sqrt(((y_hat - y_targ)**2)).sum()

In [ ]:
from scipy import spatial

CosineSimilarity = 1 - spatial.distance.cosine(y_hat, y_targ)
CosineSimilarity

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(y_targ)

In [ ]:
plt.plot(y_hat)